In [2]:
!pip3 install numpy

  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 17.3MB 107kB/s ta 0:00:01


In [2]:
import numpy as np

In [3]:
np.insert([1,1], 0, 0, 0)

array([0, 1, 1])

In [4]:
class RBM:
    def __init__(self, num_visible, num_hidden):
        self.num_hidden = num_hidden
        self.num_visible = num_visible
        self.debug_print = True
        np_rng = np.random.RandomState(1234)

        self.weights = np.asarray(np_rng.uniform(
                low  = -0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                high =  0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                size = (num_visible, num_hidden)))

        self.weights = np.insert(self.weights, 0, 0, axis = 0)
        self.weights = np.insert(self.weights, 0, 0, axis = 1)

    def train(self, data, max_epochs = 1000, learning_rate = 0.1):
        """
        Train the machine.
        Parameters
        ----------
        data: A matrix where each row is a training example consisting of the states of visible units.    
        """
        num_examples = data.shape[0]
        # Insert bias units of 1 into the first column.
        data = np.insert(data, 0, 1, axis = 1)
        for epoch in range(max_epochs):      
            pos_hidden_activations = np.dot(data, self.weights)      
            pos_hidden_probs = self._logistic(pos_hidden_activations)
            pos_hidden_probs[:,0] = 1 # Fix the bias unit.
            pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
            pos_associations = np.dot(data.T, pos_hidden_probs)
            
            neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
            neg_visible_probs = self._logistic(neg_visible_activations)
            neg_visible_probs[:,0] = 1 # Fix the bias unit.
            neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
            neg_hidden_probs = self._logistic(neg_hidden_activations)
            
            neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

        self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)
        error = np.sum((data - neg_visible_probs) ** 2)
        if self.debug_print:
            print("Epoch %s: error is %s" % (epoch, error))
    def run_visible(self, data):
    
        num_examples = data.shape[0]
        hidden_states = np.ones((num_examples, self.num_hidden + 1))
        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)
        # Calculate the activations of the hidden units.
        hidden_activations = np.dot(data, self.weights)
        # Calculate the probabilities of turning the hidden units on.
        hidden_probs = self._logistic(hidden_activations)
        # Turn the hidden units on with their specified probabilities.
        hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
        # Always fix the bias unit to 1.
        # hidden_states[:,0] = 1
        # Ignore the bias units.
        hidden_states = hidden_states[:,1:]
        return hidden_states
    # TODO: Remove the code duplication between this method and `run_visible`?
    def run_hidden(self, data):
        num_examples = data.shape[0]
        visible_states = np.ones((num_examples, self.num_visible + 1))
        # Insert bias units of 1 into the first column of data.
        data = np.insert(data, 0, 1, axis = 1)
        # Calculate the activations of the visible units.
        visible_activations = np.dot(data, self.weights.T)
        # Calculate the probabilities of turning the visible units on.
        visible_probs = self._logistic(visible_activations)
        # Turn the visible units on with their specified probabilities.
        visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)        
        visible_states = visible_states[:,1:]
        return visible_states
    def daydream(self, num_samples):
    
        samples = np.ones((num_samples, self.num_visible + 1))
        # Take the first sample from a uniform distribution.
        samples[0,1:] = np.random.rand(self.num_visible)
        for i in range(1, num_samples):
            visible = samples[i-1,:]
            hidden_activations = np.dot(visible, self.weights)      
            # Calculate the probabilities of turning the hidden units on.
        hidden_probs = self._logistic(hidden_activations)
        # Turn the hidden units on with their specified probabilities.
        hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
        # Always fix the bias unit to 1.
        hidden_states[0] = 1
        # Recalculate the probabilities that the visible units are on.
        visible_activations = np.dot(hidden_states, self.weights.T)
        visible_probs = self._logistic(visible_activations)
        visible_states = visible_probs > np.random.rand(self.num_visible + 1)
        samples[i,:] = visible_states
        # Ignore the bias units (the first column), since they're always set to 1.
        return samples[:,1:]        
    def _logistic(self, x):
        return 1.0 / (1 + np.exp(-x))


In [5]:
if __name__ == '__main__':
    r = RBM(num_visible = 6, num_hidden = 2)
    training_data = np.array([ [1,1,1,0,0,0],
                               [1,0,1,0,0,0],
                               [1,1,1,0,0,0],
                               [0,0,1,1,1,0], 
                               [0,0,1,1,0,0],
                               [0,0,1,1,1,0] ])
    r.train(training_data, max_epochs = 5000)
    print(r.weights)
    user = np.array([[0,0,0,1,1,0]])
    print(r.run_visible(user))

Epoch 4999: error is 8.984244482252368
[[ 0.05        0.00058633 -0.00100547]
 [ 0.02520172 -0.05340237  0.02059665]
 [ 0.00809188 -0.01951445  0.04015487]
 [ 0.0749431   0.07404495 -0.0145061 ]
 [ 0.02491521 -0.03804182  0.0519547 ]
 [ 0.0074316   0.07083538  0.05554719]
 [-0.02484717 -0.04949619 -0.02561542]]
[[0. 0.]]
